In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

In [ ]:
# Download nltk resources
# nltk.download('punkt')
# nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_data = pd.read_parquet('/content/drive/MyDrive/NLP_data/test-00000-of-00001.parquet')
train_data = pd.read_parquet('/content/drive/MyDrive/NLP_data/train-00000-of-00003.parquet')
validation_data = pd.read_parquet('/content/drive/MyDrive/NLP_data/validation-00000-of-00001.parquet')


In [ ]:
# Convert Parquet files to CSV
# test_data.to_csv('/content/drive/MyDrive/NLP_data/test_data.csv', index=False)
# train_data.to_csv('/content/drive/MyDrive/NLP_data/train_data.csv', index=False)
# validation_data.to_csv('/content/drive/MyDrive/NLP_data/validation_data.csv', index=False)


In [ ]:
# Load the CSV files
test_df = pd.read_csv('/content/drive/MyDrive/NLP_data/test_data.csv')
train_df = pd.read_csv('/content/drive/MyDrive/NLP_data/train_data.csv')
validation_df = pd.read_csv('/content/drive/MyDrive/NLP_data/validation_data.csv')


**Exploratory Data Analysis**

In [ ]:
print("Test Data:")
print(test_df.head())

Test Data:
                                             article  \
0  (CNN)The Palestinian Authority officially beca...   
1  (CNN)Never mind cats having nine lives. A stra...   
2  (CNN)If you've been following the news lately,...   
3  (CNN)Five Americans who were monitored for thr...   
4  (CNN)A Duke student has admitted to hanging a ...   

                                          highlights  \
0  Membership gives the ICC jurisdiction over all...   
1  Theia, a bully breed mix, was apparently hit b...   
2  Mohammad Javad Zarif has spent more time with ...   
3  17 Americans were exposed to the Ebola virus w...   
4  Student is no longer on Duke University campus...   

                                         id  
0  f001ec5c4704938247d27a44948eebb37ae98d01  
1  230c522854991d053fe98a718b1defa077a8efef  
2  4495ba8f3a340d97a9df1476f8a35502bcce1f69  
3  a38e72fed88684ec8d60dd5856282e999dc8c0ca  
4  c27cf1b136cc270023de959e7ab24638021bc43f  


In [ ]:
test_df.describe()

,article,highlights,id
count,11490,11490,11490
unique,11488,11449,11490
top,"(CNN)Tornadoes, fierce winds and severe thunde...",Investigators have released a handful of photo...,f001ec5c4704938247d27a44948eebb37ae98d01
freq,2,3,1


In [ ]:
train_df.describe()

,article,highlights,id
count,95705,95705,95705
unique,92619,91913,95705
top,(CNN) -- Conjoined Egyptian twin boys Hassan a...,This page includes the show Transcript and the...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
freq,3,83,1


In [ ]:
validation_df.describe()

,article,highlights,id
count,13368,13368,13368
unique,13368,13300,13368
top,"(CNN)Share, and your gift will be multiplied. ...",This page includes the show Transcript . Use t...,a4942dd663020ca54575471657a0af38d82897d6
freq,1,16,1


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11490 entries, 0 to 11489
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     11490 non-null  object
 1   highlights  11490 non-null  object
 2   id          11490 non-null  object
dtypes: object(3)
memory usage: 269.4+ KB


In [ ]:
# Check the shape of each dataset
print(f"Train set: {train_df.shape}")
print(f"Test set: {test_df.shape}")
print(f"Validation set: {validation_df.shape}")

Train set: (95705, 3)
Test set: (11490, 3)
Validation set: (13368, 3)


In [ ]:
# Check for null values
print(train_df.isnull().sum())

article       0
highlights    0
id            0
dtype: int64


In [ ]:
train_df['article'][0]

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

In [ ]:
train_df['highlights'][0]

"Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday . Young actor says he has no plans to fritter his cash away . Radcliffe's earnings from first five Potter films have been held in trust fund ."